In [229]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import glob
import sys 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm

In [230]:
INCLUDE_TODAY = True

In [231]:
MAX_VALID_DAY = 14 if INCLUDE_TODAY else 13
MAX_TEST_DAY = 15 if INCLUDE_TODAY else 14

In [232]:
MAX_VALID_DAY = 13
MAX_TEST_DAY = 15

In [233]:
d = pd.read_csv('../input/feed_info.csv')

In [234]:
d.head()

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,NaN,NaN,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...
2,12921,2996,15,114413 107973 117252 27736 41035 32715 125374 ...,NaN,NaN,10288.0,11354.0,NaN,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646,27077 10050 27464 20072 26018 6243 21492 14750...,NaN,NaN
3,34538,12577,18,60473 9864 8109 135558 108803 89307 37630 6481...,39654 95201 22675 49521 135558 108803 89307 37...,7446 47082 37630 99660 9844 113002 9844 113002...,24435.0,13645.0,5454;1197;3727,NaN,328;13;159;6,267 0.21298289;191 0.21298289;8 0.34298885;306...,21489 24672 2223 1882 4956 1545 10424 19791 12...,9015 21209 5026 11192 4956 1545 10424 19791 12...,20825 13395 14765 24513 7305 12837 10337 8885 ...
4,75414,7185,16,105860 4691 134820 55911 80449 79213 23233 139...,73055 66447 88970,104002 4438 104002 4438,23428.0,10730.0,17083;4123,8058;219;21639;15621;25209,81;269;159;6,269 0.86163938;81 0.86163938,25782 19583 7768 3907 14068 12681 5043 17859 1...,30483 23721 1518 22741 25118,23112 969 23112 969


In [235]:
MAX_TAGS = 14
MAX_KEYS = 18
DESC_LEN = 128
DESC_CHAR_LEN = 256
WORD_LEN = 256
START_ID = 3
UNK_ID = 1
EMPTY_ID = 2
NAN_ID = -1

In [236]:
d = d.fillna(NAN_ID)

In [237]:
d.feedid = d.feedid.astype(int)
d.authorid = d.authorid.astype(int)
d.bgm_singer_id = d.bgm_singer_id.astype(int)
d.bgm_song_id = d.bgm_song_id.astype(int)
with open('../input/word_corpus.txt', 'w') as f:
  for i, desc in enumerate(d.description.values):
    if str(desc) == str(NAN_ID):
      continue
    print(desc, file=f)
  for i, ocr in enumerate(d.ocr.values):
    if str(ocr) == str(NAN_ID):
      continue
    print(ocr, file=f)
  for i, asr in enumerate(d.asr.values):
    if str(asr) == str(NAN_ID):
      continue
    print(asr, file=f)
    
with open('../input/char_corpus.txt', 'w') as f:
  for i, desc in enumerate(d.description_char.values):
    if str(desc) == str(NAN_ID):
      continue
    print(desc, file=f)
  for i, ocr in enumerate(d.ocr_char.values):
    if str(ocr) == str(NAN_ID):
      continue
    print(ocr, file=f)
  for i, asr in enumerate(d.asr_char.values):
    if str(asr) == str(NAN_ID):
      continue
    print(asr, file=f)

In [238]:
with open('../input/author_corpus.txt', 'w') as f:
  for i, author in enumerate(d.authorid.values):
    if author == NAN_ID:
      continue
    print(author, file=f)
with open('../input/singer_corpus.txt', 'w') as f:
  for i, singer in enumerate(d.bgm_singer_id.values):
    if singer == NAN_ID:
      continue
    print(singer, file=f)
with open('../input/song_corpus.txt', 'w') as f:
  for i, song in enumerate(d.bgm_song_id.values):
    if song == NAN_ID:
      continue
    print(song, file=f)

In [239]:
with open('../input/tag_corpus.txt', 'w') as f:
  for i, tag in enumerate(d.manual_tag_list):
    if str(tag) == str(NAN_ID):
      continue
    else:
      print(str(tag).replace(';', ' '), file=f)
  for i, tag in enumerate(d.machine_tag_list):
    if str(tag) == str(NAN_ID):
      continue
    else:
      print(' '.join([x.split()[0] for x in tag.split(';')]), file=f)
    
with open('../input/key_corpus.txt', 'w') as f:
  for i, kw in enumerate(d.manual_keyword_list):
    if str(kw) == str(NAN_ID):
      continue
    else:
      print(str(kw).replace(';', ' '), file=f)
  for kw in d.machine_keyword_list:
    if str(kw) == str(NAN_ID):
      continue
    else:
      print(str(kw).replace(';', ' '), file=f)

In [240]:
d2 = pd.read_csv('../input/user_action2.csv')

In [241]:
dt = pd.read_csv('../input/test_a.csv')

In [242]:
dt2 = pd.read_csv('../input/test_b.csv')

In [243]:
dt2.head()

,userid,feedid,device
0,116490,18545,2
1,19265,53010,2
2,194374,9003,1
3,19414,75730,2
4,6434,17863,2


In [244]:
dt['date_'] = 15
dt2['date_'] = 15

In [245]:
set(d2.date_)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [246]:
# dshows = d2[['userid', 'feedid', 'date_']]
dshows = pd.concat([d2[['userid', 'feedid', 'date_']],
                  dt[['userid', 'feedid', 'date_']], 
                  dt2[['userid', 'feedid', 'date_']]])

In [247]:
# TODO add test

In [248]:
dshows.head()

,userid,feedid,date_
0,8,71474,1
1,237907,3404,1
2,237907,87814,1
3,237907,58723,1
4,82352,69580,1


In [249]:
tqdm.pandas()

In [250]:
# # 似乎不考虑date_效果更好 这个depreciate
# docs = dshows.groupby(['userid', 'date_'])['feedid'].progress_apply(list).reset_index(name='docs')
# docs.head()

In [22]:
# with open('../input/doc_corpus.txt', 'w') as f:
#   for docs_ in docs.docs:
#     print(' '.join(map(str, docs_)), file=f)

In [23]:
# docs['num_docs'] = docs.docs.apply(len)

In [24]:
# docs.num_docs.describe()

In [251]:
docs = dshows[dshows.date_ <= MAX_TEST_DAY].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='docs')
docs.head()

  0%|          | 0/20000 [00:00<?, ?it/s]

,userid,docs
0,8,"[71474, 75420, 56747, 105148, 25619, 40803, 73..."
1,12,"[102005, 17278, 44471, 89669, 59778, 35323, 21..."
2,13,"[60767, 23302, 31961, 52458, 30296, 79618, 541..."
3,25,"[52971, 103380, 70469, 8460, 42520, 2430, 2267..."
4,37,"[60130, 2505, 18148, 49725, 32160, 10705, 2185..."


In [252]:
docs_test = docs

In [253]:
# 似乎不考虑date_效果更好，好很多
with open('../input/doc_test_corpus.txt', 'w') as f:
  for docs_ in docs.docs:
    print(' '.join(map(str, docs_)), file=f)

In [254]:
docs['num_docs'] = docs.docs.apply(len)

In [255]:
docs.num_docs.describe()

count    20000.00000
mean       407.97565
std        261.52334
min         10.00000
25%        185.00000
50%        353.00000
75%        597.00000
max       1746.00000
Name: num_docs, dtype: float64

In [256]:
docs = dshows[dshows.date_ <= MAX_VALID_DAY].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='docs')
docs.head()

  0%|          | 0/19916 [00:00<?, ?it/s]

,userid,docs
0,8,"[71474, 75420, 56747, 105148, 25619, 40803, 73..."
1,12,"[102005, 17278, 44471, 89669, 59778, 35323, 21..."
2,13,"[60767, 23302, 31961, 52458, 30296, 79618, 541..."
3,25,"[52971, 103380, 70469, 8460, 42520, 2430, 2267..."
4,37,"[60130, 2505, 18148, 49725, 32160, 10705, 2185..."


In [257]:
docs_valid = docs

In [258]:
with open('../input/doc_valid_corpus.txt', 'w') as f:
  for docs_ in docs.docs:
    print(' '.join(map(str, docs_)), file=f)

In [259]:
docs['num_docs'] = docs.docs.apply(len)

In [260]:
docs.num_docs.describe()

count    19916.00000
mean       336.85710
std        228.29006
min          1.00000
25%        141.00000
50%        287.00000
75%        504.00000
max        958.00000
Name: num_docs, dtype: float64

In [261]:
users = d2[(d2.date_ < MAX_TEST_DAY) & (d2.actions > 0)].groupby(['feedid'])['userid'].progress_apply(list).reset_index(name='users')

  0%|          | 0/53962 [00:00<?, ?it/s]

In [262]:
users.head()

,feedid,users
0,1,"[233322, 6516]"
1,3,[157285]
2,4,[55610]
3,7,[117581]
4,8,"[227841, 202708, 174694, 45041, 186196, 56122,..."


In [263]:
with open('../input/user_action_test_corpus.txt', 'w') as f:
  for users_ in users.users:
    print(' '.join(map(str, users_)), file=f)

In [264]:
users['num_users'] = users.users.apply(len)

In [265]:
users.num_users.describe()

count    53962.000000
mean         9.358864
std         21.332441
min          1.000000
25%          1.000000
50%          3.000000
75%          8.000000
max        642.000000
Name: num_users, dtype: float64

In [266]:
users = d2[(d2.date_ < MAX_VALID_DAY) & (d2.actions > 0)].groupby(['feedid'])['userid'].progress_apply(list).reset_index(name='users')

  0%|          | 0/47294 [00:00<?, ?it/s]

In [267]:
with open('../input/user_action_valid_corpus.txt', 'w') as f:
  for users_ in users.users:
    print(' '.join(map(str, users_)), file=f)

In [268]:
users['num_users'] = users.users.apply(len)

In [269]:
users.num_users.describe()

count    47294.000000
mean         9.044699
std         20.584722
min          1.000000
25%          1.000000
50%          3.000000
75%          8.000000
max        601.000000
Name: num_users, dtype: float64

In [270]:
# TODO doc 对应的user比较多 特别复赛20w user， 初赛只有2w 可能需要shuffle 大的doc 然后限定小窗口
users = dshows[dshows.date_ <= MAX_TEST_DAY].groupby(['feedid', 'date_'])['userid'].progress_apply(list).reset_index(name='users')

  0%|          | 0/511773 [00:00<?, ?it/s]

In [271]:
with open('../input/user_test_corpus.txt', 'w') as f:
  for users_ in users.users:
    print(' '.join(map(str, users_)), file=f)

In [272]:
users['num_users'] = users.users.apply(len)

In [273]:
users.num_users.describe()

count    511773.000000
mean         15.943618
std          44.222516
min           1.000000
25%           1.000000
50%           4.000000
75%          13.000000
max        2503.000000
Name: num_users, dtype: float64

In [274]:
users = dshows[dshows.date_ <= MAX_VALID_DAY].groupby(['feedid', 'date_'])['userid'].progress_apply(list).reset_index(name='users')

  0%|          | 0/427376 [00:00<?, ?it/s]

In [275]:
with open('../input/user_valid_corpus.txt', 'w') as f:
  for users_ in users.users:
    print(' '.join(map(str, users_)), file=f)

In [276]:
users['num_users'] = users.users.apply(len)

In [277]:
users.num_users.describe()

count    427376.00000
mean         15.69776
std          42.16033
min           1.00000
25%           1.00000
50%           4.00000
75%          13.00000
max        1820.00000
Name: num_users, dtype: float64

In [278]:
feeds = {}
def cache_feed():
  df = pd.read_csv('../input/feed_info.csv')
  df = df.fillna(-1)
  for row in tqdm(df.itertuples(), total=len(df), ascii=True, desc='feed_info'):
    row = row._asdict()               
    feeds[row['feedid']] = row
cache_feed()

feed_info:   0%|          | 0/106444 [00:00<?, ?it/s]

In [279]:
docs_test.num_docs.max()

1746

In [280]:
docs_valid.num_docs.max()

958

In [281]:
# 下面对应的都是valid
docs = docs_valid
with open('../input/author_valid_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    authors = [feeds[doc]['authorid'] for doc in docs_]
    print(' '.join(map(str, authors)), file=f)

  0%|          | 0/19916 [00:00<?, ?it/s]

In [282]:
with open('../input/singer_valid_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    singers = [feeds[doc]['bgm_singer_id'] for doc in docs_]
    print(' '.join(map(str, singers)), file=f)

  0%|          | 0/19916 [00:00<?, ?it/s]

In [283]:
with open('../input/song_valid_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    songs = [feeds[doc]['bgm_song_id'] for doc in docs_]
    print(' '.join(map(str, songs)), file=f)

  0%|          | 0/19916 [00:00<?, ?it/s]

In [284]:
# with open('../input/tag_valid_corpus.txt', 'w') as f:
#   for docs_ in tqdm(docs.docs):
#     tags = [str(feeds[doc]['manual_tag_list']).replace(';', ' ') for doc in docs_]
#     print(' '.join(map(str, tags)), file=f)
#     tags2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_tag_list']).split(';')]) for doc in docs_]
#     print(' '.join(map(str, tags2)), file=f)

In [285]:
# with open('../input/key_valid_corpus.txt', 'w') as f:
#   for docs_ in tqdm(docs.docs):
#     keys = [str(feeds[doc]['manual_keyword_list']).replace(';', ' ') for doc in docs_]
#     print(' '.join(map(str, keys)), file=f)
#     keys2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_keyword_list']).split(';')]) for doc in docs_]
#     print(' '.join(map(str, keys2)), file=f)

In [286]:
with open('../input/tag_valid_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    tags = [str(feeds[doc]['manual_tag_list']).replace(';', ' ') for doc in docs_]
    tags = set(' '.join(tags).split())
    print(' '.join(tags), file=f)
    tags2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_tag_list']).split(';')]) for doc in docs_]
    tags2 = set(' '.join(tags2).split())
    print(' '.join(tags2), file=f)

  0%|          | 0/19916 [00:00<?, ?it/s]

In [287]:
with open('../input/key_valid_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    tags = [str(feeds[doc]['manual_keyword_list']).replace(';', ' ') for doc in docs_]
    tags = set(' '.join(tags).split())
    print(' '.join(tags), file=f)
    tags2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_keyword_list']).split(';')]) for doc in docs_]
    tags2 = set(' '.join(tags2).split())
    print(' '.join(tags2), file=f)

  0%|          | 0/19916 [00:00<?, ?it/s]

In [288]:
docs = docs_test

In [289]:
with open('../input/author_test_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    authors = [feeds[doc]['authorid'] for doc in docs_]
    print(' '.join(map(str, authors)), file=f)
with open('../input/singer_test_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    singers = [feeds[doc]['bgm_singer_id'] for doc in docs_]
    print(' '.join(map(str, singers)), file=f)
with open('../input/song_test_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    songs = [feeds[doc]['bgm_song_id'] for doc in docs_]
    print(' '.join(map(str, songs)), file=f)

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [290]:
with open('../input/tag_test_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    tags = [str(feeds[doc]['manual_tag_list']).replace(';', ' ') for doc in docs_]
    tags = set(' '.join(tags).split())
    print(' '.join(tags), file=f)
    tags2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_tag_list']).split(';')]) for doc in docs_]
    tags2 = set(' '.join(tags2).split())
    print(' '.join(tags2), file=f)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [291]:
with open('../input/key_test_corpus.txt', 'w') as f:
  for docs_ in tqdm(docs.docs):
    tags = [str(feeds[doc]['manual_keyword_list']).replace(';', ' ') for doc in docs_]
    tags = set(' '.join(tags).split())
    print(' '.join(tags), file=f)
    tags2 = [' '.join([x.split()[0] for x in str(feeds[doc]['machine_keyword_list']).split(';')]) for doc in docs_]
    tags2 = set(' '.join(tags2).split())
    print(' '.join(tags2), file=f)

  0%|          | 0/20000 [00:00<?, ?it/s]